## Limpieza dataset 2015
A continuación se limpia y analiza los datos correspondientes al dataset del año 2015. Este dataset se compone de las siguientes columnas:
- X: longitud
- Y: latitud
- FID: Número de identificación del registro
- Ubicación: Nombre de la dirección completa donde ocurrió el siniestro
- Fallecidos: Cantidad de personas fallecidas
- Graves: Cantidad de personas en estado grave
- MenosGrave: Cantidad de personas en estado menos grave
- Leve: Cantidad de personas con heridas leves
- Accidentes: Cantida de accidentes

<hr>

El objetivo para este dataset es:
- Eliminar aquellas columnas que no aportan datos relevantes o que no coincidan con las estructuras de los otros dataset.
- Extraer los valores de Calle1, Calle2, Número y Comuna a partir de la columna Ubicación.
- Normalizar los datos del dataset (por ejemplo: letras a mayúsculas y sin tildes o imputación de valores nulos)
- Exportar el dataset normalizado.

In [1]:
# Importacion de librerias que se emplearan
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import re

# Configuracion de pandas para que no bloque asignaciones masivas
pd.options.mode.chained_assignment = None

# Lectura del dataset y previsualizacion de los datos
df = pd.read_excel("../data/siniestros atropello RM2015.xlsx")
df.head()

,X,Y,FID,Ubicacion,Fallecidos,Graves,MenosGrave,Leve,Accidentes
0,-70.798508,-33.364435,1,"CAMINO RENCA LAMPA & CAMINO AGUAS CLARAS, PUDA...",0,1,0,0,1
1,-70.797142,-33.354941,2,"CAMINO LO ECHEVERS & CAMINO PRIVADO, QUILICURA",0,1,0,0,1
2,-70.797944,-33.511716,3,"EL CONQUISTADOR & SENADORA MARIA DE LA CRUZ, M...",0,1,0,1,1
3,-70.797199,-33.513151,4,"EL CONQUISTADOR & PINTOR HORACIO GARCIA, MAIPU",0,1,0,0,1
4,-70.797021,-33.518601,5,"1370 EL CONQUISTADOR, MAIPU",0,1,0,0,1


In [2]:
df.describe()

,X,Y,FID,Fallecidos,Graves,MenosGrave,Leve,Accidentes
count,1841.000000,1841.000000,1841.000000,1841.000000,1841.000000,1841.000000,1841.000000,1841.000000
mean,-70.642630,-33.473420,921.000000,0.057577,0.295492,0.098316,0.828897,1.190114
std,0.066754,0.067868,531.595241,0.248798,0.529177,0.310334,0.768646,0.553379
min,-70.798508,-33.631695,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,-70.691135,-33.522422,461.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,-70.640550,-33.454299,921.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,-70.586695,-33.425596,1381.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,-70.493987,-33.325531,1841.000000,3.000000,4.000000,2.000000,7.000000,8.000000


## Datos básicos del dataset

In [3]:
print("===================================================================")
print("Tamaño del dataframe:")
print(df.shape)
print("===================================================================")
print("Tipo de variable de las columnas:")
print(df.dtypes)
print("===================================================================")
print("Cantidad de valores nulos por columna:")
print(df.isna().sum())
print("===================================================================")

Tamaño del dataframe:
(1841, 9)
Tipo de variable de las columnas:
X             float64
Y             float64
FID             int64
Ubicacion      object
Fallecidos      int64
Graves          int64
MenosGrave      int64
Leve            int64
Accidentes      int64
dtype: object
Cantidad de valores nulos por columna:
X             0
Y             0
FID           0
Ubicacion     0
Fallecidos    0
Graves        0
MenosGrave    0
Leve          0
Accidentes    0
dtype: int64


Para las columnas que son de tipo string, se determina el número de carácteres del texto más corto, texto más largo y el número de carácteres. Esto se hará para las siguientes columnas:
- Ubicacion

<hr>

Primero se deja en nulo aquellas columnas cuya cadena de caracteres sea un string vacío.

In [4]:
# Funcion para reemplazar por nulo los string vacios
def replace_null(col):
    if(col is not np.nan):
        if(len(col.strip()) == 0):
            return np.nan
        else:
            return col
    else:
        return np.nan

In [6]:
df["Ubicacion"] = df["Ubicacion"].apply(replace_null)

# Se vuelve a mostrar la cantidad de nulos resultantes
df.isna().sum()

X             0
Y             0
FID           0
Ubicacion     0
Fallecidos    0
Graves        0
MenosGrave    0
Leve          0
Accidentes    0
dtype: int64

In [7]:
# Funcion para obtener la mayor y menor cantidad de carácteres y la cantidad promedio de carácteres
def LargoCadena(lista):
    # Acumuladores y registros para las variables a determinar
    _min = 0
    _palabraMin = ""
    _max = 0
    _palabraMax = ""
    _i = 0
    _suma = 0
    _promedio = 0
    
    for e in lista:
        if(e is not np.nan):# elemento no nulo
            if(_i == 0):# Primer registro
                _min = len(e.strip())
                _max = len(e.strip())
                _palabraMin = e.strip()
                _palabraMax = e.strip()
                
            # Acumulador de sumas para calcular promedio posterior
            _suma = _suma + len(e.strip())
            
            # Re asignacion para largos maximo y minimo 
            if(len(e.strip()) < _min):
                _min = len(e.strip())
                _palabraMin = e.strip()
                
            if(len(e.strip()) > _max):
                _max = len(e.strip())
                _palabraMax = e.strip()
            
            # Contador para el calculo de promedio
            _i = _i + 1
            
    # Calculo del promedio fuera del bucle for
    _promedio = _suma / _i
    
    # Se muestra un breve reporte con la informacion
    print("====================================================================")
    print("Palabra más larga:")
    print(_palabraMax)
    print("Número de carácteres:")
    print(_max)
    print("====================================================================")
    print("Palabra más corta:")
    print(_palabraMin)
    print("Número de carácteres:")
    print(_min)
    print("====================================================================")
    print("Cantidad promedio de carácteres:")
    print(int(_promedio))
    print("Número de palabras evaluadas:")
    print(_i)
    print("====================================================================")

In [8]:
LargoCadena(df["Ubicacion"])

Palabra más larga:
PRESIDENTE EDUARDO FREI MONTALVA & CENTRAL CARDENAL SILVA HENRIQUEZ, LO ESPEJO
Número de carácteres:
78
Palabra más corta:
126 OSSA, NUNOA
Número de carácteres:
15
Cantidad promedio de carácteres:
38
Número de palabras evaluadas:
1841


## Normalización de textos

In [9]:
def normalizar_texto(texto):
    if(texto is not np.nan):
        texto = texto.upper()
        texto = re.sub("`", "", texto)
        texto = re.sub(r"Á|Ä|Â|À", "A", texto)
        texto = re.sub(r"É|Ë|Ê|È", "E", texto)
        texto = re.sub(r"Í|Ï|Î|Ì", "I", texto)
        texto = re.sub(r"Ó|Ö|Ô|Ò", "O", texto)
        texto = re.sub(r"Ú|Ü|Û|Ù", "U", texto)
        return texto
    else:
        return np.nan
    
df["Ubicacion"] = df["Ubicacion"].apply(normalizar_texto)
df.head()

,X,Y,FID,Ubicacion,Fallecidos,Graves,MenosGrave,Leve,Accidentes
0,-70.798508,-33.364435,1,"CAMINO RENCA LAMPA & CAMINO AGUAS CLARAS, PUDA...",0,1,0,0,1
1,-70.797142,-33.354941,2,"CAMINO LO ECHEVERS & CAMINO PRIVADO, QUILICURA",0,1,0,0,1
2,-70.797944,-33.511716,3,"EL CONQUISTADOR & SENADORA MARIA DE LA CRUZ, M...",0,1,0,1,1
3,-70.797199,-33.513151,4,"EL CONQUISTADOR & PINTOR HORACIO GARCIA, MAIPU",0,1,0,0,1
4,-70.797021,-33.518601,5,"1370 EL CONQUISTADOR, MAIPU",0,1,0,0,1


## Separación de la columna Ubicación
A partir de la columna "Ubicacion" se pueden obtener los valores para Calle1, Calle2, Numero y Comuna. Esto porque la estructura de esta columna se organiza de estas dos formas:
- Caso 1: {Calle1} & {Calle2}, {Comuna}
- Caso 2: {Numero} {Calle1}, {Comuna} <br>

Por lo cual se emplean expresiones regulares para separar los valores y almacenarlos en nuvas columnas.

In [10]:
# Funcion para separar numero, calle, comuna
def Direccion_formato1(direccion, campo):
    altura = re.search(r"\d{1,100}\s", direccion)
    calle1 = re.search(r"\s\w(\w|\s)*\,", direccion)
    comuna = re.search(r"\,\s\w(\w|\s)*", direccion)    
    if(altura != None):
        altura = altura.group(0).strip()
    else:
        altura = 0
    
    if(calle1 != None):
        calle1 = calle1.group(0).replace(",", "").strip()
    else:
        calle1 = ""
    
    if(comuna != None):
        comuna = comuna.group(0).replace(",", "").strip()
    else:
        comuna = ""
    
    direccion = {
        "altura": altura,
        "calle1": calle1,
        "calle2": "",
        "comuna": comuna
    }
    
    return direccion[campo]

# Funcion para separar calle1, calle2, comuna
def Direccion_formato2(direccion, campo):
    calle1 = re.search(r"\w(\w|\s)*\&", direccion)
    calle2 = re.search(r"\&\s\w(\w|\s)*\,", direccion)
    comuna = re.search(r"\,\s\w(\w|\s)*", direccion)

    if(calle1 != None):
        calle1 = calle1.group(0).replace("&", "").strip()
    else:
        calle1 = ""
    
    if(calle2 != None):
        calle2 = calle2.group(0).replace("&", "").replace(",", "").strip()
    else:
        calle2 = ""
    
    if(comuna != None):
        comuna = comuna.group(0).replace(",", "").strip()
    else:
        comuna = ""
    
    direccion = {
        "calle1": calle1,
        "calle2": calle2,
        "comuna": comuna,
        "altura": 0
    }
    
    return direccion[campo]


def Descomposicion_Direccion(direccion, campo):
    if("&" not in direccion):# Caso 2: calle1 + numero + comuna
        return Direccion_formato1(direccion, campo)
    else:# Caso1: calle1 + calle2 + comuna
        return Direccion_formato2(direccion, campo)
    
df["Comuna"] = df["Ubicacion"].apply(Descomposicion_Direccion, campo = "comuna")
df["Calle1"] = df["Ubicacion"].apply(Descomposicion_Direccion, campo = "calle1")
df["Calle2"] = df["Ubicacion"].apply(Descomposicion_Direccion, campo = "calle2")
df["Numero"] = df["Ubicacion"].apply(Descomposicion_Direccion, campo = "altura")
df.columns

Index(['X', 'Y', 'FID', 'Ubicacion', 'Fallecidos', 'Graves', 'MenosGrave',
       'Leve', 'Accidentes', 'Comuna', 'Calle1', 'Calle2', 'Numero'],
      dtype='object')

In [12]:
df["Comuna"].value_counts()

SANTIAGO               190
PUENTE ALTO            148
LAS CONDES             141
PROVIDENCIA            120
MAIPU                  109
NUNOA                   77
SAN BERNARDO            76
LA FLORIDA              73
PUDAHUEL                72
ESTACION CENTRAL        68
PENALOLEN               59
QUINTA NORMAL           58
EL BOSQUE               55
RECOLETA                50
VITACURA                45
CERRO NAVIA             44
MACUL                   44
LA GRANJA               39
CONCHALI                37
LO BARNECHEA            35
QUILICURA               31
LA REINA                31
RENCA                   28
SAN RAMON               27
LA CISTERNA             25
LO ESPEJO               23
HUECHURABA              22
LO PRADO                21
LA PINTANA              21
SAN JOAQUIN             20
INDEPENDENCIA           18
SAN MIGUEL              18
PEDRO AGUIRRE CERDA     16
Name: Comuna, dtype: int64

Se puede apreciar que las comunas de Peñalolen y Ñuñoa aparecen repetidas con diferentes nombres o que están mal escritas; se cambian por los nombres correctos.

In [13]:
# Normalizacion de los nombres de comunas
df["Comuna"] =  df["Comuna"].apply(lambda x: "PEÑALOLEN" if x == "PENALOLEN" else x)
df["Comuna"] =  df["Comuna"].apply(lambda x: "ÑUÑOA" if x == "NUNOA" else x)
df["Comuna"].value_counts()

SANTIAGO               190
PUENTE ALTO            148
LAS CONDES             141
PROVIDENCIA            120
MAIPU                  109
ÑUÑOA                   77
SAN BERNARDO            76
LA FLORIDA              73
PUDAHUEL                72
ESTACION CENTRAL        68
PEÑALOLEN               59
QUINTA NORMAL           58
EL BOSQUE               55
RECOLETA                50
VITACURA                45
CERRO NAVIA             44
MACUL                   44
LA GRANJA               39
CONCHALI                37
LO BARNECHEA            35
QUILICURA               31
LA REINA                31
RENCA                   28
SAN RAMON               27
LA CISTERNA             25
LO ESPEJO               23
HUECHURABA              22
LO PRADO                21
LA PINTANA              21
SAN JOAQUIN             20
INDEPENDENCIA           18
SAN MIGUEL              18
PEDRO AGUIRRE CERDA     16
Name: Comuna, dtype: int64

## Eliminacion de columnas innecesarias o que no coinciden con los otros dataset

In [14]:
# Eliminacion FID
del df["FID"]
# Eliminacion Accidentes
del df["Accidentes"]
# Eliminacion Ubicacion
del df["Ubicacion"]
df.head()

,X,Y,Fallecidos,Graves,MenosGrave,Leve,Comuna,Calle1,Calle2,Numero
0,-70.798508,-33.364435,0,1,0,0,PUDAHUEL,CAMINO RENCA LAMPA,CAMINO AGUAS CLARAS,0
1,-70.797142,-33.354941,0,1,0,0,QUILICURA,CAMINO LO ECHEVERS,CAMINO PRIVADO,0
2,-70.797944,-33.511716,0,1,0,1,MAIPU,EL CONQUISTADOR,SENADORA MARIA DE LA CRUZ,0
3,-70.797199,-33.513151,0,1,0,0,MAIPU,EL CONQUISTADOR,PINTOR HORACIO GARCIA,0
4,-70.797021,-33.518601,0,1,0,0,MAIPU,EL CONQUISTADOR,,1370


## Creación de la columna si fue en intersección el accidente y columna año
Esta nueva columna estará llena de 1's y 0's en función de si ocurrió o no en una intersección respectivamente. Esto se calcula en función de los valores de las columnas Call1, Calle2 y Numero.

In [15]:
def Interseccion(numero):
    if(numero == 0):
        return 1
    else:
        return 0

df["Interseccion"] = df["Numero"].apply(Interseccion)
df["Anio"] = 2015
df.head()

,X,Y,Fallecidos,Graves,MenosGrave,Leve,Comuna,Calle1,Calle2,Numero,Interseccion,Anio
0,-70.798508,-33.364435,0,1,0,0,PUDAHUEL,CAMINO RENCA LAMPA,CAMINO AGUAS CLARAS,0,1,2015
1,-70.797142,-33.354941,0,1,0,0,QUILICURA,CAMINO LO ECHEVERS,CAMINO PRIVADO,0,1,2015
2,-70.797944,-33.511716,0,1,0,1,MAIPU,EL CONQUISTADOR,SENADORA MARIA DE LA CRUZ,0,1,2015
3,-70.797199,-33.513151,0,1,0,0,MAIPU,EL CONQUISTADOR,PINTOR HORACIO GARCIA,0,1,2015
4,-70.797021,-33.518601,0,1,0,0,MAIPU,EL CONQUISTADOR,,1370,0,2015


## Se renombran las columnas
Se dejan las columnas con los nombres de: X, Y, Anio, Comuna, Calle1, Numero, Interseccion, Fallecidos, Graves, MenosGraves y Leves

In [16]:
nombres_col = {
    "MenosGrave": "MenosGraves",
    "Leve": "Leves"
}
df = df.rename(columns = nombres_col)
df.columns

Index(['X', 'Y', 'Fallecidos', 'Graves', 'MenosGraves', 'Leves', 'Comuna',
       'Calle1', 'Calle2', 'Numero', 'Interseccion', 'Anio'],
      dtype='object')

## Se reordenan las columnas a la siguiente secuencia:
- 1° X
- 2° Y
- 3° Anio
- 4° Comuna
- 5° Calle1
- 6° Calle2
- 7° Numero
- 8° Interseccion
- 9° Fallecidos
- 10° Graves
- 11° MenosGraves
- 12° Leves

In [17]:
df = df[[
    "X", 
    "Y", 
    "Anio", 
    "Comuna", 
    "Calle1", 
    "Calle2",
    "Numero", 
    "Interseccion", 
    "Fallecidos", 
    "Graves",
    "MenosGraves",
    "Leves"]]
df.columns

Index(['X', 'Y', 'Anio', 'Comuna', 'Calle1', 'Calle2', 'Numero',
       'Interseccion', 'Fallecidos', 'Graves', 'MenosGraves', 'Leves'],
      dtype='object')

## Se exporta el nuevo dataset

In [18]:
df_copia = df
df_copia.to_csv("../data_limpia/Dataset_2015_ordenado.csv", index = False)